<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Retrieval_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Retrieval evaluation

Just to evaluate the models using precision and recall...

## Question list (30)
Have the relevant document for each so you can measure, recall, precision, F! score

|Doc_ID|Name|Relevant_Q|
|---|---|---|
|1|Evidence & insights - reducing household food waste & plastic packaging (packaged vs loose)|1,2,3,4,5,6,7,14,19|
|2|Food waste collection guidance |8, 17|
|3|Food Waste Reduction Roadmap - Executive Summary|9, 14, 16,19|
|4,19|Food Waste Reduction Roadmap - Hospitality|9|
|5,19|Food Waste Reduction Roadmap - Manufacturers|9|
|6,19|Food Waste Reduction Roadmap - Primary Producers|9,10|
|7,19|Food Waste Reduction Roadmap - Retail|9,11|
|8|Identifying impacts from food and farm digestates|12,13|
|9|Industry Guidance - Dealing with Household Food Waste at AD Facilities - Management of Liners|14,8, 15|
|10|Literature review - relationship between household food waste collection and food waste prevention|8, 15, 16, 17, 18, 19|
|11|The food waste reduction roadmap progress report 2022|9, 19|
|12|Towards the 2030 Food Waste Commitment -- setting our coalition baseline|14, 19|
|13|Upscaling farm food waste measurement in the UK|19, 22|
|14|WRAP-Evidence Review Plastic Packaging and Fresh Produce 171218|3, 4, 14|

1. "How should I store bananas?"
2. "What temperature should I keep milk?"
3. "How do I increase shelf life of lettuce?
4. "How does selling fresh produce loose, affect waste?"
5. "What is a best-before date for?"
6. "What is a use-by date for?"
7. "How does removing date labels help?"
8. "What are the reasons to not participate in a food waste scheme?"
9. "What is the UN's sustainable development goal with respect to food?"
10. "Which document relates to primary producers and food waste?"
11. "Which document relates to retailers and food waste?"
12. "What is the most common use for digestate?"
13. "What affects the costs of valorisation?"
14. "How many tonnes of food waste are there each year?"
15. "What is the benefit of caddy liners?"
16. "How to prevent food loss?"
17. "How to prevent food waste?"
18. "How does composting affect food waste?"
19. "What are the main drivers of food waste? got to here!"
20. "Online shopping and food waste"
21. "What is involved with Courthold 2030?"
22. "How do you measure in field food waste?"

In [ ]:
!pip install -Uq sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 32.5 MB/s eta 0:00:00


In [ ]:
import sqlite3
from sentence_transformers import SentenceTransformer, util
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# model used to encode question (should be same as the one used to create database)
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1')
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# Question
# documents have been indexed from 0 so document 1 = 0 etc
keys = list(range(1,15))
# answers (relevant questions)
values = [
    [1,3,4,5,6,7,14,19],
    [8,17],
    [9,14, 16],
    [9,19],
    [9,19],
    [9,10, 19],
    [9,11, 19],
    [12,13, 19],
    [14,8,15],
    [2, 8,15,16,17,18,19],
    [9, 19],
    [14, 19],
    [19, 22],
    [1,3,4,14],

]

QA = dict(zip(keys,values))

In [ ]:
# question list
Questions = [
        "How should I store bananas?",
        "What temperature should I keep milk?",
        "How do I increase shelf life of lettuce?",
        "How does selling fresh produce loose, affect waste?",
        "What is a best-before date for?",
        "What is a use-by date for?",
        "How does removing date labels help?",
        "What are the reasons to not participate in a food waste scheme?",
        "What is the UN's sustainable development goal with respect to food?",
        "Which document relates to primary producers and food waste?",
        "Which document relates to retailers and food waste?",
        "What is the most common use for digestate?",
        "What affects the costs of valorisation?",
        "How many tonnes of food waste are there each year?",
        "What is the benefit of caddy liners?",
        "How to prevent food loss?",
        "How to prevent food waste?",
        "How does composting affect food waste?",
        "What are the main drivers of food waste?",
        "Online shopping and food waste",
        "What is involved with Courthold 2030?",
        "How do you measure in field food waste?"
]

In [ ]:
# '/content/drive/MyDrive/Knowledge-database-10-4-24-mpnet.db' ; /content/drive/MyDrive/Knowledge-database-10-4-24-All-mini-L6-v2.db
DB = '/content/drive/MyDrive/Knowledge-database-10-4-24-mixedbread-large-v1.db'


In [ ]:
def score_retrieval(question=["No question"],
                    model=None, db=DB):
    relevant = []
    q = model.encode(question)
    conn = sqlite3.connect(db)

    # Create a cursor object
    cursor = conn.cursor()
    # load row by row
    cursor.execute('SELECT * FROM documents')

    for row in cursor:
        score  = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()
        # if meets the score threshold and is not already in the list
        # Vary based on sentence size for single sentence have 0.7 for large context maybe 0.5?
        # 100 token = 0.6
        if score >= 0.5:
            relevant.append((row[1]))

    # return only unique relevant documents.
    return list(set(relevant))

In [ ]:
# For sense checking

def find_top_k_relevance(question=["No question"], model=None, n=20, db=DB):
  q = model.encode(question)
  scores = {}
  conn = sqlite3.connect(db)

# Create a cursor object
  cursor = conn.cursor()
  # load row by row
  cursor.execute('SELECT * FROM documents')
  for row in cursor:
    scores[row[0]] = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

  return dict(sorted(scores.items(), key = lambda x: x[1], reverse = True)[:n])


find_top_k_relevance(question =["Which document relates to food waste and retailers?"], model=model)

{1401: array([[0.77326477]], dtype=float32),
 6417: array([[0.76573944]], dtype=float32),
 401: array([[0.7620941]], dtype=float32),
 7937: array([[0.7581463]], dtype=float32),
 989: array([[0.7565614]], dtype=float32),
 1103: array([[0.7537936]], dtype=float32),
 5265: array([[0.75352687]], dtype=float32),
 6770: array([[0.74947965]], dtype=float32),
 6646: array([[0.7478083]], dtype=float32),
 1365: array([[0.7477835]], dtype=float32),
 5679: array([[0.74453175]], dtype=float32),
 513: array([[0.74336594]], dtype=float32),
 5788: array([[0.7433449]], dtype=float32),
 5833: array([[0.7433449]], dtype=float32),
 5870: array([[0.7433449]], dtype=float32),
 6226: array([[0.7433449]], dtype=float32),
 714: array([[0.7427078]], dtype=float32),
 4979: array([[0.74202347]], dtype=float32),
 6113: array([[0.7420212]], dtype=float32),
 6549: array([[0.74189144]], dtype=float32)}

In [ ]:
import pprint
conn = sqlite3.connect(DB)
cursor = conn.cursor()
cursor.execute("""SELECT * FROM documents
                 WHERE id = 3701""")
rows = cursor.fetchone()
conn.close()
pprint.pprint(f"{rows[3]} context: {rows[2]}, {rows[1]}")

('Ehlert, P. et al. (2019c) context: 08 - Identifying impacts from food and '
 'farm digestates.pdf, 8')


In [ ]:
import os
DIR = "/content/drive/MyDrive/WRAP food reports"

In [ ]:
%%timeit
## get scores for the document retrieval
def scoring(q, qa=QA):
    precision , recall, f1 = 0,0,0
    # AS questions don't have a zeroeth q need to start from q + 1
    q_no = q + 1
    # true positive
    all_relevant = []
    for k, v in QA.items():
        if q_no in v:
            all_relevant.append(k)
    correct = 0
    for i in retrieved_docs:
        if i in all_relevant:
            correct +=1
    # no relevant docs recieved - all scores are 0
    if len(all_relevant) == 0:
        return (precision, recall, f1)
    else:
        # if there are no documents that meet our criteria
        if len(retrieved_docs) == 0:
            return (precision, recall, f1)
        else:
            precision = abs(len(set(all_relevant) & set(retrieved_docs))) / abs(len(retrieved_docs))
            # recall
            recall = abs(len(set(all_relevant) & set(retrieved_docs))) / abs(len(all_relevant))
            # avoid divide by 0 error
            if precision == 0 or recall == 0:
                f1 = 0.0
            # if the other values are not zero
            else:
                f1 = (2*precision*recall)/(precision + recall)

    return (precision, recall, f1)


# loop to get all metrics
p , r , f = [], [], []

for i in range(len(Questions)):
        q = Questions[i]
        print(q)
        retrieved_docs = score_retrieval(question = q, model=model)
        print(retrieved_docs)
        score_ = scoring(Questions.index(q))
        print(score_)
        p.append(score_[0])
        r.append(score_[1])
        f.append(score_[2])

How should I store bananas?
[1, 2, 5, 7, 8, 9, 10, 13, 14]
(0.2222222222222222, 1.0, 0.3636363636363636)
What temperature should I keep milk?
[1, 8, 9, 13, 14]
(0.0, 0.0, 0.0)
How do I increase shelf life of lettuce?
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14]
(0.15384615384615385, 1.0, 0.2666666666666667)
How does selling fresh produce loose, affect waste?
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(0.14285714285714285, 1.0, 0.25)
What is a best-before date for?
[1, 2, 5, 8, 9, 10, 12, 13, 14]
(0.1111111111111111, 1.0, 0.19999999999999998)
What is a use-by date for?
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(0.07142857142857142, 1.0, 0.13333333333333333)
How does removing date labels help?
[1, 2, 5, 8, 9, 10, 11, 12, 13, 14]
(0.1, 1.0, 0.18181818181818182)
What are the reasons to not participate in a food waste scheme?
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(0.21428571428571427, 1.0, 0.35294117647058826)
What is the UN's sustainable development goal with respect to

In [ ]:
import pandas as pd

df = pd.DataFrame({"Question":Questions, "Precision":p, "Recall":r, "F1":f})


In [ ]:
df

,Question,Precision,Recall,F1
0,How should I store bananas?,1.000000,1.0,1.000000
1,What temperature should I keep milk?,0.000000,0.0,0.000000
2,How do I increase shelf life of lettuce?,1.000000,1.0,1.000000
3,"How does selling fresh produce loose, affect w...",0.142857,1.0,0.250000
4,What is a best-before date for?,1.000000,1.0,1.000000
5,What is a use-by date for?,1.000000,1.0,1.000000
6,How does removing date labels help?,1.000000,1.0,1.000000
7,What are the reasons to not participate in a f...,0.214286,1.0,0.352941
8,What is the UN's sustainable development goal ...,0.545455,1.0,0.705882
9,Which document relates to primary producers an...,0.071429,1.0,0.133333


In [ ]:
#name with db sentence transformer and method of sentence splitting
df.to_csv("/content/drive/MyDrive/all-mini-context-100-added-doc-summaries.csv")